#### Глава 4 Выделение и использование лингвистических признаков

Лингвистические признаки -- это, например, теги частей речи, синтаксические
зависимости и именованне сущности. 

<b>В этой главе</b> вы научитесь выделять и генерировать текст на основе тегов частей речи и синтаксических зави- симостей, что позволит создавать отвечающие на вопросы чат-боты,
находить в тексте конкретные фразы и многое другое.

<b>Выделение и генерация текста с помощью тегов частей речи</b>

In [1]:
import ru_core_news_lg
import spacy as sp
import sys

from spacy.symbols import ORTH, LEMMA
from IPython.display import Image
from spacy.tokens.doc import Doc
from spacy.vocab import Vocab

from spacy import displacy
from IPython.display import Image

nlp = sp.load('ru_core_news_lg')


Для начала выделим из токенов признаки общих частей речи и увидим, как spaCy распознает различные части речи:


In [2]:
doc = nlp(u"Фильм собрал $1.5 миллиона в 2019 году.")
for token in doc:
    print('{0:10s} {1:10s} {2:10s}'.format(token.text, token.pos_, sp.explain(token.pos_)))

Фильм      NOUN       noun      
собрал     VERB       verb      
$          SYM        symbol    
1.5        NUM        numeral   
миллиона   NOUN       noun      
в          ADP        adposition
2019       ADJ        adjective 
году       NOUN       noun      
.          PUNCT      punctuation


In [3]:
print('{0:10s} {1:10s} {2:10s} {3:10s}'.format('text', 'pos_', 'dep_', 'explain'))
print('_'* 45)
for token in doc:
    print('{0:10s} {1:10s} {2:10s} {3:10s}'.format(token.text, token.pos_, token.dep_, sp.explain(token.tag_)))

# ВОПРОС, почему совпадают pos_ and tag_??? Не знаю, но метод dep_ отлично заменяет его

text       pos_       dep_       explain   
_____________________________________________
Фильм      NOUN       nsubj      noun      
собрал     VERB       ROOT       verb      
$          SYM        obj        symbol    
1.5        NUM        nummod     numeral   
миллиона   NOUN       nummod:gov noun      
в          ADP        case       adposition
2019       ADJ        amod       adjective 
году       NOUN       obl        noun      
.          PUNCT      punct      punctuation


Теперь сравним теги общих и уточненных частей речи для того же предложения,

<b>Выделение описаний денежных сумм</b>

In [4]:
doc = nlp(u"Фильм собрал $1.5 миллиона в 2019 году.")

phrase = ''
for token in doc:
    if token.tag_ == 'SYM':
        phrase = token.text
        i = token.i + 1
        while doc[i].dep_ == 'nummod' or doc[i].dep_ == 'nummod:gov':
            phrase += doc[i].text + ' '
            i += 1
        break
phrase = phrase[:-1]
print(phrase)


$1.5 миллиона


<b>Преобразование утвердительных высказываний в вопросительные</b>

In [5]:
doc = nlp(u'Я хочу купить то свежее яблоко.')

print('{0:10s} {1:10s} {2:10s} {3:10s}'.format('text', 'pos_', 'dep_', 'explain'))
print('_'* 45)
for token in doc:
    print('{0:10s} {1:10s} {2:10s} {3:10s}'.format(token.text, token.pos_, token.dep_, sp.explain(token.pos_)))

text       pos_       dep_       explain   
_____________________________________________
Я          PRON       nsubj      pronoun   
хочу       VERB       ROOT       verb      
купить     VERB       xcomp      verb      
то         DET        det        determiner
свежее     ADJ        amod       adjective 
яблоко     NOUN       obj        noun      
.          PUNCT      punct      punctuation


In [6]:
Image(url="images/picture_3.png", width=500, height=300)

Так как эта схема подхожит только для англйиского языка, то мы отойдем от нее и адаптируем под русский язык.

In [7]:
doc = nlp(u'Я хочу купить то свежее яблоко.')
print(doc)

sent = ''
for i, token in enumerate(doc):
    if token.dep_ == 'nsubj' and token.text == 'Я':
        sent = doc[:i].text + 'Вы ' + doc[i + 1:].text
        break
print(sent)

# К этому моменту предложение должно выглядеть так:
# вы хочу купить свежее яблоко.

doc = nlp(sent)
for i, token in enumerate(doc):
    if token.dep_ == 'det' and token.text == 'то':
        sent = doc[:i].text + ' это ' + doc[i + 1:].text
        break
print(sent)
# К этому моменту предложение должно выглядеть так:
# Вы хочу купить это свежее яблоко.

doc = nlp(sent)
for i, token in enumerate(doc):
    if token.dep_ == 'ROOT':
        sent = doc[:i].text + ' хотите ' + doc[i + 1:].text
        break
print(sent)

# К этому моменту предложение должно выглядеть так:
# Вы хотите купить это свежее яблоко.
doc = nlp(sent)
sent = doc[:len(doc)-1].text + '?'

# Наконец, мы получаем: Вы хотите купить это свежее яблоко?
print(sent)

Я хочу купить то свежее яблоко.
Вы хочу купить то свежее яблоко.
Вы хочу купить это свежее яблоко.
Вы хотите купить это свежее яблоко.
Вы хотите купить это свежее яблоко?


In [8]:
Image(url="images/picture_4.png", width=500, height=300)

In [9]:
def replace_token(doc, tok_dep, tok_text, repl):
    sent = ''
    for i, token in enumerate(doc):
        if token.dep_ == tok_dep and token.text == tok_text:
            sent = doc[:i].text + ' ' + repl + ' ' + doc[i + 1:].text
            break
    return nlp(sent)
    
doc = nlp(u'Я хочу купить то свежее яблоко.')  
print(doc)
doc = replace_token(doc, 'nsubj', 'Я', 'Вы') 
print(doc)

Я хочу купить то свежее яблоко.
 Вы хочу купить то свежее яблоко.


#### Использование меток синтаксических зависимостей при обработке текста

Различаем подлежащие и дополнения

In [10]:
Image(url="images/picture_5.png", width=500, height=300)
# ВАЖНО - в русском tag_ и pos_ одно и то же, то есть нет уточнений частей речи

In [11]:
# dep_
Image(url="images/picture.png", width=500, height=300)

In [12]:
doc = nlp(u'Таня выучила все теоремы по математическому анализу вчера и объяснила их Коле.')

# dep_ == метки зависимости
print('{0:15s} {1:10s} {2:10s} {3:10s} {4:10s}'.format('text', 'pos_', 'dep_', 'tag_', 'explain'))
print('_'* 55)
for token in doc:
    print('{0:15s} {1:10s} {2:10s} {3:10s} {4:10s}'.format(token.text, token.pos_, token.dep_, token.tag_, sp.explain(token.dep_)))

text            pos_       dep_       tag_       explain   
_______________________________________________________
Таня            PROPN      nsubj      PROPN      nominal subject
выучила         VERB       ROOT       VERB       root      
все             DET        det        DET        determiner
теоремы         NOUN       obj        NOUN       object    
по              ADP        case       ADP        case marking
математическому ADJ        amod       ADJ        adjectival modifier
анализу         NOUN       nmod       NOUN       modifier of nominal
вчера           ADV        advmod     ADV        adverbial modifier
и               CCONJ      cc         CCONJ      coordinating conjunction
объяснила       VERB       conj       VERB       conjunct  
их              PRON       obj        PRON       object    
Коле            PROPN      iobj       PROPN      indirect object
.               PUNCT      punct      PUNCT      punctuation


Имена "Таня" и "Коле" совпадают в pos_, но отличаются в dep_
text            pos_       dep_       tag_       explain   
____________________________________________________________
Таня            PROPN      nsubj      PROPN      nominal subject   
Коле            PROPN      iobj       PROPN      indirect object

#### Выясняем, какой вопрос должен задать чат-бот

Иногда для извлечения необходимой информации приходится обходить дерево зависимостей предложения.

In [13]:
Image(url="images/picture_6.png", width=300, height=100)

In [14]:
Image(url="images/picture_7.png", width=500, height=150)

In [15]:
# Возвращение прямого дополнения (по книжке это dobj, однако в русском spacy есть только obj, iobj)
def find_chunk(doc):
    chunk = ''
    for i, token in enumerate(doc):
        if token.dep_ == 'obj':
            shift = len([w for w in token.children])
            #print([w for w in token.children])
            chunk = doc[i-shift:i+1]
            break
    return chunk

def determine_question_type(chunk):
    question_type = 'yesno'
    for token in chunk:
        # ищет модификатор прилагательного (пример: "красный" шар)
        if token.dep_ == 'amod':
            question_type = 'info'
    return question_type


def generate_question(doc, question_type):
    sent = ''

    for i, token in enumerate(doc):
        if token.dep_ == 'nsubj' and token.text == 'Я':
            sent = doc[:i].text + 'Вы ' + doc[i + 1:].text
            break
        
    doc = nlp(sent)
    for i, token in enumerate(doc):
        if token.dep_ == 'ROOT':
            sent = doc[:i].text + ' хотите ' + doc[i + 1:].text
            break
        
    doc = nlp(sent)
    if question_type == 'info':
        for i, token in enumerate(doc):
            if token.dep_ == 'obj':
                sent = 'Хотите ли ' + doc[:i].text  +  doc[i+1:].text
                break
    if question_type == 'yesno':
        for i, token in enumerate(doc):
            if token.dep_ == 'obj':
                sent = doc[:i-1].text + ' хотите зеленое ' + doc[i:].text
                break

    doc = nlp(sent)
    sent = doc[:len(doc)-1].text + '?'        

    return sent


In [16]:
doc = nlp(u"Я хочу грушу.")
# dep_ == метки зависимости
print('{0:15s} {1:10s} {2:10s} {3:10s} {4:10s}'.format('text', 'pos_', 'dep_', 'tag_', 'explain'))
print('_'* 55)
for token in doc:
    print('{0:15s} {1:10s} {2:10s} {3:10s} {4:10s}'.format(token.text, token.pos_, token.dep_, token.tag_, sp.explain(token.dep_)))

text            pos_       dep_       tag_       explain   
_______________________________________________________
Я               PRON       nsubj      PRON       nominal subject
хочу            VERB       ROOT       VERB       root      
грушу           NOUN       obj        NOUN       object    
.               PUNCT      punct      PUNCT      punctuation


In [17]:
# Возвращение первого прямого дополнения 
print(find_chunk(doc))

грушу


In [18]:
# пробуем использовать модель чат бота
chunk = find_chunk(doc)
if str(chunk) == '':
  print('The sentence does not contain a direct object.')
  sys.exit()
question_type = determine_question_type(chunk)
question = generate_question(doc, question_type)
print(question)

Вы хотите зеленое грушу?
